<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_10_03_Using_Tools_with_Conversations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit to call tools within a prompted conversation.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [32]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [33]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [34]:
from typing import List, Optional

async def create_anthropic_specification(model: enums.AnthropicModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    if conversation_id is not None:
        _ = await graphlit.client.delete_conversation(conversation_id)

async def prompt_conversation(conversation_id: str, prompt: str, tools: List[input_types.ToolDefinitionInput]):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id, None, tools)

        role = response.prompt_conversation.message.role if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None

        if role != enums.ConversationRoleTypes.ASSISTANT:
            print(f'Unexpected message role [{role}].')
            return None

        return response.prompt_conversation.message.tool_calls if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def continue_conversation(conversation_id: str, responses: List[input_types.ConversationToolResponseInput]):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.continue_conversation(conversation_id, responses)

        return response.continue_conversation.message.message if response.continue_conversation is not None and response.continue_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)


In [35]:
import json
from typing import Literal

def get_stock_price(ticker: str, currency: Literal["USD", "EUR"] = "USD"):
    """
    Get the stock price of a company, by ticker symbol

    Parameters
    ----------
    ticker
        The ticker symbol of the company
    currency
        The currency to use
    """
    return f"182.41 {currency}, -0.48 (0.26%) today"

def call_function(tool_call):
    function_name = tool_call.name
    arguments = json.loads(tool_call.arguments)

    response_content = None

    try:
        # Use globals() to get the function by name and call it with dynamic arguments
        response_content = globals()[function_name](**arguments)

        print(f'Function response: {response_content}')
    except KeyError:
        print(f"Function '{function_name}' not found.")
    except TypeError as e:
        print(f"Error calling function: {e}")

    return response_content

Execute Graphlit example

In [39]:
import time
from IPython.display import display, Markdown, HTML

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

tool_calls = None
conversation_id = None

#specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET)
specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_MINI_128K)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id)

    if conversation_id is not None:
        print(f'Created conversation [{conversation_id}].')

        tool_name = "get_stock_price"
        tool_description = "Get the stock price of a company, by ticker symbol"
        tool_schema = """
        {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The ticker symbol of the company"
                },
                "currency": {
                    "type": "string",
                    "description": "The currency to use",
                    "enum": [
                        "USD",
                        "EUR"
                    ],
                    "default": "USD"
                }
            },
            "required": [
                "ticker"
            ]
        }
        """

        tools = [input_types.ToolDefinitionInput(name=tool_name, description=tool_description, schema=tool_schema)]

        prompt = "What are Microsoft and Nvidia's stock prices today?"

        tool_calls = await prompt_conversation(conversation_id, prompt, tools)

        if tool_calls is not None:
            display(Markdown('### Tool Calls:'))

            for tool_call in tool_calls:
                if tool_call is not None and tool_call.id is not None:
                    display(Markdown(f'### Tool [{tool_call.id}]:'))
                    display(Markdown(f'**Name:**\n{tool_call.name}'))
                    display(Markdown(f'**Arguments:**\n{tool_call.arguments}'))
                    print()

Deleted all conversations and specifications.
Created specification [a221f24f-1828-4899-9508-7a23dcce91dc].
Created conversation [6d8ec602-7920-4513-828b-914deb0860df].


### Tool Calls:

### Tool [call_LfJgvfwVPhUqIKGaRSdSsK1L]:

**Name:**
get_stock_price

**Arguments:**
{
  "ticker": "MSFT"
}

### Tool [call_yxcmenRherqML5HwxSOZX4L4]:

**Name:**
get_stock_price

**Arguments:**
{
  "ticker": "NVDA"
}

In [40]:
import json

if conversation_id is not None and tool_calls is not None:
    responses = []

    for tool_call in tool_calls:
        if tool_call is not None and tool_call.id is not None:
            response_content = call_function(tool_call)

            if response_content is not None:
                responses.append(input_types.ConversationToolResponseInput(id=tool_call.id, content=response_content))

    message = await continue_conversation(conversation_id, responses)

    if message is not None:
        display(Markdown('### Conversation:'))
        display(Markdown(f'**User:**\n{prompt}'))
        display(Markdown(f'**Assistant:**\n{message}'))
        print()


Function response: 182.41 USD, -0.48 (0.26%) today
Function response: 182.41 USD, -0.48 (0.26%) today


### Conversation:

**User:**
What are Microsoft and Nvidia's stock prices today?

**Assistant:**
Microsoft's stock price is 182.41 USD, down 0.48 (0.26%) today.

Nvidia's stock price is also 182.41 USD, down 0.48 (0.26%) today.